##### Copyright &copy; 2020 The TensorFlow Authors.

<font size=-1>Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  See the License for the specific language governing permissions and limitations under the License.</font>

# Create a TFX pipeline using templates

## Introduction

This document will provide instructions to create a TensorFlow Extended (TFX) pipeline
using *templates* which are provided with TFX Python package.
Many of instructions are Linux shell commands, which will run on an AI Platform Notebooks instance. Corresponding Jupyter Notebook code cells which invoke those commands using `!` are provided.

You will build a pipeline using [Taxi Trips dataset](
https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew)
released by the City of Chicago. We strongly encourage you to try building
your own pipeline using your dataset by utilizing this pipeline as a baseline.


## Step 1. Set up your environment.

AI Platform Pipelines will prepare a development environment to build a pipeline, and a Kubeflow Pipeline cluster to run the newly built pipeline.

**NOTE:** To select a particular TensorFlow version, or select a GPU instance, create a TensorFlow pre-installed instance in AI Platform Notebooks.

**NOTE:** There might be some errors during package installation. For example: 

>"ERROR: some-package 0.some_version.1 has requirement other-package!=2.0.,&lt;3,&gt;=1.15, but you'll have other-package 2.0.0 which is incompatible." Please ignore these errors at this moment.


Install `tfx`, `kfp`, and `skaffold`, and add installation path to the `PATH` environment variable.

In [1]:
requirements_file = 'requirements.txt'

In [5]:
%%writefile {requirements_file}

tfx==0.21.0
kfp==0.2.5

Writing requirements.txt


In [6]:
# Install tfx and kfp Python packages.
!pip install --user --upgrade -q -r requirements.txt
# Download skaffold and set it executable.
!curl -Lo skaffold https://storage.googleapis.com/skaffold/releases/latest/skaffold-linux-amd64 && chmod +x skaffold && mv skaffold /home/jupyter/.local/bin/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 37.3M  100 37.3M    0     0  75.0M      0 --:--:-- --:--:-- --:--:-- 74.9M


In [7]:
# Set `PATH` to include user python binary directory and a directory containing `skaffold`.
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

env: PATH=/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


Let's check the versions of TFX.

In [8]:
!python -c "import tfx; print('TFX version: {}'.format(tfx.__version__))"

TFX version: 0.21.0


In AI Platform Pipelines, TFX is running in a hosted Kubernetes environment using [Kubeflow Pipelines](https://www.kubeflow.org/docs/pipelines/overview/pipelines-overview/).

Let's set some environment variables to use Kubeflow Pipelines.

First, get your GCP project ID.

In [10]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GCP_PROJECT_ID=shell_output[0]
print("GCP project ID:" + GCP_PROJECT_ID)

GCP project ID:mlops-workshop


We also need to access your KFP cluster. You can access it in your Google Cloud Console under "AI Platform > Pipeline" menu. The "endpoint" of the KFP cluster can be found from the URL of the Pipelines dashboard, or you can get it from the URL of the Getting Started page where you launched this notebook. Let's create an `ENDPOINT` environment variable and set it to the KFP cluster endpoint. **ENDPOINT should contain only the hostname part of the URL.** For example, if the URL of the KFP dashboard is `https://1e9deb537390ca22-dot-asia-east1.pipelines.googleusercontent.com/#/start`, ENDPOINT value becomes `1e9deb537390ca22-dot-asia-east1.pipelines.googleusercontent.com`.

>**NOTE: You MUST set your ENDPOINT value below.**

In [11]:
# This refers to the KFP cluster endpoint
ENDPOINT='b408c7cc27aa8bb-dot-us-central2.pipelines.googleusercontent.com' # Enter your ENDPOINT here.
if not ENDPOINT:
    from absl import logging
    logging.error('Set your ENDPOINT in this cell.')

Set the image name as `tfx-pipeline` under the current GCP project.

In [12]:
# Docker image name for the pipeline image 
CUSTOM_TFX_IMAGE='gcr.io/' + GCP_PROJECT_ID + '/cifar-tfx-image'

And, it's done. We are ready to create a pipeline.

## Step 2. Build the pipeline

Set the pipeline's compile settings and the default values for the pipeline's runtime parameters.

In [28]:
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT_ID=shell_output[0]

GCP_REGION='us-central1'
CUSTOM_TFX_IMAGE='gcr.io/' + GCP_PROJECT_ID + '/cifar-tfx-image'
ARTIFACT_STORE_URI='gs://env55-artifact-store'
MODEL_NAME='cifar10-classifier'

In [29]:
%env KUBEFLOW_TFX_IMAGE={CUSTOM_TFX_IMAGE}
%env ARTIFACT_STORE_URI={ARTIFACT_STORE_URI}
%env GCP_REGION={GCP_REGION}
%env MODEL_NAME={MODEL_NAME}

env: KUBEFLOW_TFX_IMAGE=gcr.io/mlops-workshop/cifar-tfx-image
env: ARTIFACT_STORE_URI=gs://env55-artifact-store
env: GCP_REGION=us-central1
env: MODEL_NAME=cifar10-classifier


In [30]:
!tfx pipeline compile --engine kubeflow --pipeline_path pipeline/runner.py

CLI
Compiling pipeline
cifar10_continuous_training
/home/jupyter/.local/lib/python3.7/site-packages/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '
Pipeline compiled successfully.
Pipeline package path: /home/jupyter/mlops-labs/examples/tfx-cifar/cifar10_continuous_training.tar.gz


TFX includes the `taxi` template with the TFX python package. If you are planning to solve a point-wise prediction problem, including classification and regresssion, this template could be used as a starting point.

The `tfx template copy` CLI command copies predefined template files into your project directory.

Change the working directory context in this notebook to the project directory.

In [32]:
!tfx pipeline create  \
--pipeline_path=pipeline/runner.py \
--endpoint={ENDPOINT} \
--build_target_image={CUSTOM_TFX_IMAGE}

CLI
Creating pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Reading build spec from build.yaml
Target image gcr.io/mlops-workshop/cifar-tfx-image is not used. If the build spec is provided, update the target image in the build spec file build.yaml.
Use skaffold to build the container image.
/home/jupyter/.local/bin/skaffold
New container image is built. Target image is available in the build spec file.
cifar10_continuous_training
/home/jupyter/.local/lib/python3.7/site-packages/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '
Pipeline compiled successfully.
Pipeline package path: /home/jupyter/mlops-labs/examples/tfx-cifar/cifar10_continuous_training.tar.gz
{'created_at': datetime.datetime(2020, 3, 6, 20, 21, 14, tzinfo=tzlocal()),
 'default_version': {'code_source_url': None,
                     'c

If you need you can delete the pipeline with the following command.

In [31]:
!tfx pipeline delete --pipeline_name {PIPELINE_NAME} --endpoint {ENDPOINT}

CLI
Deleting pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Pipeline cifar10_continuous_training deleted successfully.



A pipeline definition file for [argo](https://argoproj.github.io/argo/) will be generated, too. The name of this file is `${PIPELINE_NAME}.tar.gz`. For example, it will be `my_pipeline.tar.gz` if the name of your pipeline is `my_pipeline`. It is recommended NOT to include this pipeline definition file into source control, because it will be generated from other Python files and will be updated whenever you update the pipeline. For your convenience, this file is already listed in `.gitignore` which is generated automatically.

NOTE: `kubeflow` will be automatically selected as an orchestration engine if `airflow` is not installed and `--engine` is not specified.

Now start an execution run with the newly created pipeline using the `tfx run create` command.

In [33]:
!tfx run create --pipeline_name={PIPELINE_NAME} --endpoint={ENDPOINT}

CLI
Creating a run for pipeline: cifar10_continuous_training
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Run created for pipeline: cifar10_continuous_training
+-----------------------------+--------------------------------------+----------+---------------------------+
| pipeline_name               | run_id                               | status   | created_at                |
+=============================+======================================+==========+===========================+
| cifar10_continuous_training | 3f3e3a11-7c67-44ac-9932-d61acc45599d |          | 2020-03-06T20:22:00+00:00 |
+-----------------------------+--------------------------------------+----------+---------------------------+


Or, you can also run the pipeline in the KFP Dashboard.  The new execution run will be listed under Experiments in the KFP Dashboard.  Clicking into the experiment will allow you to monitor progress and visualize the artifacts created during the execution run.

However, we recommend visiting the KFP Dashboard. You can access the KFP Dashboard from the Cloud AI Platform Pipelines menu in Google Cloud Console. Once you visit the dashboard, you will be able to find the pipeline, and access a wealth of information about the pipeline.
For example, you can find your runs under the *Experiments* menu, and when you open your execution run under Experiments you can find all your artifacts from the pipeline under *Artifacts* menu.

>Note: If your pipeline run fails, you can see detailed logs for each TFX component in the Experiments tab in the KFP Dashboard.
    
One of the major sources of failure is permission related problems. Please make sure your KFP cluster has permissions to access Google Cloud APIs. This can be configured [when you create a KFP cluster in GCP](https://cloud.google.com/ai-platform/pipelines/docs/setting-up), or see [Troubleshooting document in GCP](https://cloud.google.com/ai-platform/pipelines/docs/troubleshooting).

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Alternatively, you can clean up individual resources by visiting each consoles:
- [Google Cloud Storage](https://console.cloud.google.com/storage)
- [Google Container Registry](https://console.cloud.google.com/gcr)
- [Google Kubernetes Engine](https://console.cloud.google.com/kubernetes)
